# Tutorial CS003: Solar Forecasting with TCN

This tutorial demonstrates how to use Temporal Convolutional Networks (TCN) for solar power generation forecasting. You will learn how to:
1. Handle solar generation intermittency and zero-values during night.
2. Implement a TCN with dilated convolutions and residual connections.
3. Implement chausal convolutions for time series.

In [ ]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

sys.path.append(os.path.abspath('../../src'))
from models.cnn import TCNModel
from preprocessing.pipeline import DataPipeline
from utils.metrics import evaluate_all
from utils.plotting import plot_forecast, set_style
from utils.trainer import Trainer
from data.download import download_public_dataset

set_style()

In [ ]:
data_path = download_public_dataset('D005', save_dir='../../data/raw')
df = pd.read_csv(data_path)
pipeline = DataPipeline()
train_df, val_df, test_df = pipeline.split_data(df)

target_col = 'generation'
train_scaled = pipeline.fit_transform(train_df, [target_col])
val_scaled = pipeline.transform(val_df, [target_col])
test_scaled = pipeline.transform(test_df, [target_col])

WINDOW_SIZE = 168 # One week of hourly data
X_train, y_train = pipeline.create_sequences(train_scaled, WINDOW_SIZE)
X_val, y_val = pipeline.create_sequences(val_scaled, WINDOW_SIZE)
X_test, y_test = pipeline.create_sequences(test_scaled, WINDOW_SIZE)

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()), batch_size=32)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()), batch_size=32)

In [ ]:
input_size = 1
num_channels = [16, 32, 64]
model = TCNModel(input_size, num_channels, kernel_size=3)
trainer = Trainer(model, torch.nn.MSELoss())
trainer.train(train_loader, val_loader, epochs=5, save_path='tcn_best.pth')

In [ ]:
y_pred_scaled = trainer.predict(test_loader)
y_pred = pipeline.scaler.inverse_transform(y_pred_scaled)
y_true = pipeline.scaler.inverse_transform(y_test)

plot_forecast(y_true[:168], y_pred[:168], title="TCN Solar Power Forecast", ylabel="Generation (MW)", save_path='../../figures/cs003_tcn_forecast.pdf')